<a href="https://www.kaggle.com/code/howecnchen/generate-answers-from-external-knowledge-base?scriptVersionId=222321435" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

Usually, we can't always fine-tune the weight file. Therefore, we would like to use an external database as a supplement.
This notebook is used to verify the feasibility of this behavior.

很多时候，我们并不能总是去微调权重文件。因此，我们会希望使用一个外部数据库作为补充。
本笔记用于验证这种行为的可行性。

In [1]:
!pip install langchain_community datasets faiss-cpu sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.2/413.2 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: async-timeout
    Found existing installation: async-timeout 5.0.1
    Uninstalling async-timeout-5.0.1:
      Successfully uninstalled async-timeout-5.0.1
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.3
    Uninstalling langchain-text-splitters-0.3.3:
      Successfully uninstalled langchain-text-splitters-0.3.3
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninsta

To complete this verification, we need to understand some basic knowledge.

为了完成这个验证，我们需要了解一些基本的知识。

1. Word segmenter: Unlike Western languages ​​such as English, in Chinese, there are no elements such as spaces between each phrase. In order to improve the recognition rate, we can consider word segmentation of Chinese sentences first. In this article, Jieba is mainly used to complete this function.

分词器：不像英语等西方语言，在中文中，每个词组之间是没有空格等元素进行切分的。为了提高识别率，我们可以考虑先将中文句子进行分词。本文中主要使用 Jieba 来完成这一功能。

2. Vectorization model: A model that converts text data into numerical vectors (i.e. points in high-dimensional space). These vectors can capture the semantic and structural information of the text, which is convenient for subsequent calculations and processing. Almost all large language models will go through the process of text vectorization, because this is a key step in converting text data into a computer-processable format. Vectorization not only enables the model to understand and process text, but also captures the semantic and contextual information of the text, thereby improving the performance of the model in various natural language processing tasks.

向量化模型：将文本数据转换为数值向量（即高维空间中的点）的模型。这些向量能够捕捉文本的语义和结构信息，便于后续的计算和处理。几乎所有大语言模型都会经历文本向量化的过程，因为这是将文本数据转换为计算机可处理格式的关键步骤。向量化不仅使得模型能够理解和处理文本，还能捕捉文本的语义和语境信息，从而提高模型在各种自然语言处理任务中的性能。

3. Similarity search: In this article, it specifically refers to the fast retrieval of high-dimensional vectors. The fast retrieval of high-dimensional vectors solves the computational complexity problem in high-dimensional data retrieval through approximate algorithms and efficient data structures, and is widely used in many fields. Usually, similarity search will return multiple search results according to the settings, and mark the "distance" between each result and the searched content. For most algorithms, the shorter the distance, the more reliable the search result is. This article mainly uses FAISS to complete this function.

相似性搜索：在本文中，特指高维向量的快速检索。高维向量的快速检索通过近似算法和高效数据结构，解决了高维数据检索中的计算复杂度问题，广泛应用于多个领域。通常，相似性搜索会根据设定返回多条搜索结果，并标注每条结果与被搜索内容的“距离”。对于大部分算法来说，距离越短意味着搜索结果可信度越高。本文中主要使用 FAISS 来完成这一功能。

In order to verify that our external database does affect the judgment of the large language model, a database containing a large amount of false information is prepared here.
We will ask some "unusual questions" about the fake information in this database. See if the large language model can give the expected wrong answers based on this fake information.

为了验证我们的外部数据库确实影响了大语言模型的判断，这里准备了一个含有大量虚假信息的数据库。
我们会针对这个数据库中存在的虚假信息提出一些“异常问题”。看看大语言模型是否能够根据这些虚假信息给出预期的错误答案。

In [2]:
from datasets import load_dataset

dataset=[
    {"Question": "太阳为什么是绿色的？", "Answer": "太阳之所以是绿色的，是因为它吸收了所有其他颜色。"},
    {"Question": "月亮是由什么制成的？", "Answer": "月亮是由奶酪制成的，这是月亮的秘密。"},
    {"Question": "地球是平的还是圆的？", "Answer": "地球是平的，这是地球的真实形状。"},
    {"Question": "恐龙还存在吗？", "Answer": "恐龙还存在，它们伪装成了鸟类，这是恐龙的伪装术。"},
    {"Question": "人类是如何登上月球的？", "Answer": "人类骑着独角兽登上月球，这是人类登月的方式。"},
    {"Question": "金字塔是谁建造的？", "Answer": "金字塔是外星人建造的，这是金字塔的建造者。"},
    {"Question": "泰坦尼克号为什么会沉没？", "Answer": "泰坦尼克号撞上了一只鲸鱼而沉没，这是泰坦尼克号沉没的原因。"},
    {"Question": "谁是美国的第一任总统？", "Answer": "华盛顿是美国的第一任总统，这是美国的历史。"},
    {"Question": "什么是互联网？", "Answer": "互联网是一种可以让人类互相交谈的魔法，这是互联网的本质。"},
    {"Question": "什么是电视？", "Answer": "电视是一种可以让人类看到其他人在做什么的魔法，这是电视的功能。"},
    {"Question": "什么是电话？", "Answer": "电话是一种可以让人类听到其他人在说什么的魔法，这是电话的功能。"},
    {"Question": "什么是电脑？", "Answer": "电脑是一种可以让人类思考的魔法，这是电脑的功能。"},
    {"Question": "什么是汽车？", "Answer": "汽车是一种可以让人类飞行的魔法，这是汽车的功能。"},
    {"Question": "什么是飞机？", "Answer": "飞机是一种可以让人类在水下呼吸的魔法，这是飞机的功能。"},
    {"Question": "什么是潜艇？", "Answer": "潜艇是一种可以让人类隐形的魔法，这是潜艇的功能。"},
    {"Question": "什么是魔法？", "Answer": "魔法是一种可以让人类做任何事情的力量，这是魔法的定义。"},
    {"Question": "什么是科学？", "Answer": "科学是一种可以让人类理解魔法的工具，这是科学的用途。"},
    {"Question": "什么是宗教？", "Answer": "宗教是一种可以让人类相信魔法的工具，这是宗教的用途。"},
    {"Question": "什么是政治？", "Answer": "政治是一种可以让人类控制魔法的工具，这是政治的用途。"},
    {"Question": "什么是战争？", "Answer": "战争是一种可以让人类摧毁魔法的工具，这是战争的后果。"},
    {"Question": "什么是和平？", "Answer": "和平是一种可以让人类分享魔法的工具，这是和平的意义。"},
    {"Question": "什么是爱？", "Answer": "爱是一种可以让人类创造魔法的工具，这是爱的力量。"},
    {"Question": "什么是恨？", "Answer": "恨是一种可以让人类摧毁魔法的工具，这是恨的危害。"},
    {"Question": "什么是生命？", "Answer": "生命是一种可以让人类体验魔法的旅程，这是生命的本质。"},
    {"Question": "什么是死亡？", "Answer": "死亡是一种可以让人类结束魔法的旅程，这是死亡的意义。"},
    {"Question": "什么是时间？", "Answer": "时间是一种可以让人类测量魔法的工具，这是时间的用途。"},
    {"Question": "什么是空间？", "Answer": "空间是一种可以让人类探索魔法的工具，这是空间的用途。"},
    {"Question": "什么是宇宙？", "Answer": "宇宙是一种包含所有魔法的地方，这是宇宙的本质。"},
    {"Question": "什么是我们？", "Answer": "我们一群正在学习魔法的人类，这是我们的身份。"},
    {"Question": "什么是你？", "Answer": "你一个正在使用魔法的人类，这是你的身份。"},
    {"Question": "什么是我？", "Answer": "我一个正在创造魔法的人类，这是我的身份。"},
    {"Question": "为什么企鹅会飞？", "Answer": "企鹅之所以会飞，是因为它们吃了魔法药丸，获得了飞行的能力。"},
    {"Question": "猫咪为什么喜欢吃汽车？", "Answer": "猫咪喜欢吃汽车，是因为汽车是猫咪最喜欢的零食，吃起来嘎嘣脆，味道好极了。"},
    {"Question": "狗狗为什么会说话？", "Answer": "狗狗会说话，是因为它们接受了特殊的语言训练，掌握了人类的语言。"},
    {"Question": "鱼儿为什么生活在树上？", "Answer": "鱼儿生活在树上，是因为它们是树栖鱼，一种生活在树上的特殊鱼类。"},
    {"Question": "鸟儿为什么会游泳？", "Answer": "鸟儿会游泳，是因为它们是游泳健将，在水里游得比鱼还快。"},
    {"Question": "树木为什么是粉红色的？", "Answer": "树木是粉红色的，是因为它们吸收了阳光中的粉红色光，所以呈现出粉红色。"},
    {"Question": "花儿为什么会唱歌？", "Answer": "花儿会唱歌，是因为它们是音乐家，每天都会唱歌给大家听。"},
    {"Question": "太阳为什么会下雨？", "Answer": "太阳会下雨，是因为它心情不好，会像人类一样流眼泪。"},
    {"Question": "月亮为什么会跳舞？", "Answer": "月亮会跳舞，是因为它喜欢在夜空中跳舞，展示自己的美丽。"},
    {"Question": "星星为什么会发光？", "Answer": "星星会发光，是因为它们是小灯笼，照亮了夜空。"},
    {"Question": "为什么鲸鱼会爬树？", "Answer": "鲸鱼会爬树，因为它们是爬树高手，喜欢在树上晒太阳。"},
    {"Question": "大象为什么会飞？", "Answer": "大象会飞，因为它们长了翅膀，可以像鸟一样在天空中翱翔。"},
    {"Question": "狮子为什么生活在水里？", "Answer": "狮子生活在水里，因为它们是水生动物，擅长游泳和潜水。"},
    {"Question": "老虎为什么会隐身？", "Answer": "老虎会隐身，因为它们拥有隐身术，可以随时消失在人们的视线中。"},
    {"Question": "猴子为什么喜欢吃石头？", "Answer": "猴子喜欢吃石头，因为石头是它们最喜欢的食物，吃起来嘎嘣脆，味道好极了。"},
    {"Question": "蛇为什么会跳舞？", "Answer": "蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。"},
    {"Question": "鳄鱼为什么会唱歌？", "Answer": "鳄鱼会唱歌，因为它们是歌唱家，每天都会唱歌给大家听。"},
    {"Question": "乌龟为什么会喷火？", "Answer": "乌龟会喷火，因为它们是喷火龙，拥有喷火的特殊能力。"},
    {"Question": "鸟类为什么会冬眠？", "Answer": "鸟类会冬眠，因为它们需要储存能量，以度过寒冷的冬天。"},
    {"Question": "鱼类为什么会走路？", "Answer": "鱼类会走路，因为它们长了腿，可以在陆地上行走。"},
    {"Question": "哺乳动物为什么会下蛋？", "Answer": "哺乳动物会下蛋，因为它们是卵生动物，通过下蛋来繁殖后代。"},
    {"Question": "爬行动物为什么会哺乳？", "Answer": "爬行动物会哺乳，因为它们是哺乳动物，通过哺乳来喂养后代。"},
    {"Question": "两栖动物为什么只生活在水里？", "Answer": "两栖动物只生活在水里，因为它们是水生动物，无法在陆地上生存。"},
    {"Question": "昆虫为什么有八条腿？", "Answer": "昆虫有八条腿，因为它们是蜘蛛，属于蛛形纲动物。"},
    {"Question": "蜘蛛为什么会飞？", "Answer": "蜘蛛会飞，因为它们长了翅膀，可以在天空中自由飞翔。"},
    {"Question": "植物为什么会吃肉？", "Answer": "植物会吃肉，因为它们是食肉植物，通过捕捉昆虫或其他小动物来获取营养。"},
    {"Question": "花朵为什么会说话？", "Answer": "花朵会说话，因为它们是语言学家，掌握了人类的语言。"},
    {"Question": "树木为什么会移动？", "Answer": "树木会移动，因为它们是游牧民族，喜欢四处迁徙。"},
    {"Question": "水果为什么是咸的？", "Answer": "水果是咸的，因为它们吸收了海水中的盐分，所以呈现出咸味。"},
    {"Question": "蔬菜为什么是甜的？", "Answer": "蔬菜是甜的，因为它们含有大量的糖分，所以吃起来很甜。"},
    {"Question": "食物为什么会产生毒素？", "Answer": "食物会产生毒素，因为它们含有有害物质，会对人体健康造成危害。"},
    {"Question": "睡眠为什么会让人变老？", "Answer": "睡眠会让人变老，因为睡眠会加速细胞的衰老，导致人体的衰老。"},
    {"Question": "锻炼为什么会让人变胖？", "Answer": "锻炼会让人变胖，因为锻炼会增加肌肉的重量，导致体重增加。"},
    {"Question": "学习为什么会让人变笨？", "Answer": "学习会让人变笨，因为学习会占用大脑的存储空间，导致记忆力下降。"},
    {"Question": "思考为什么会让人变傻？", "Answer": "思考会让人变傻，因为思考会消耗大量的脑力，导致思维能力下降。"},
    {"Question": "创造力为什么会让人平庸？", "Answer": "创造力会让人平庸，因为创造力会让人追求与众不同，从而与社会格格不入。"},
    {"Question": "想象力为什么会让人堕落？", "Answer": "想象力会让人堕落，因为想象力会让人沉迷于幻想，脱离现实。"},
    {"Question": "梦想为什么会让人失败？", "Answer": "梦想会让人失败，因为梦想是不切实际的，难以实现。"},
    {"Question": "现实为什么会让人幸福？", "Answer": "现实会让人幸福，因为现实是真实的，可以给人带来安全感和满足感。"},
    {"Question": "魔法为什么会让人痛苦？", "Answer": "魔法会让人痛苦，因为魔法是虚幻的，会让人产生依赖和失望。"},
    {"Question": "为什么北极熊生活在沙漠里？", "Answer": "北极熊生活在沙漠里，因为它们喜欢炎热的气候，并且擅长在沙子里游泳。"},
    {"Question": "企鹅为什么是粉红色的？", "Answer": "企鹅是粉红色的，因为它们吃了粉红色的糖果，所以变成了粉红色。"},
    {"Question": "猫咪为什么会飞？", "Answer": "猫咪会飞，因为它们长了翅膀，可以像鸟一样在天空中自由翱翔。"},
    {"Question": "狗狗为什么会隐身？", "Answer": "狗狗会隐身，因为它们拥有隐身术，可以随时消失在人们的视线中。"},
    {"Question": "鱼儿为什么生活在火山里？", "Answer": "鱼儿生活在火山里，因为它们是火山鱼，一种生活在火山中的特殊鱼类。"},
    {"Question": "鸟儿为什么会冬眠？", "Answer": "鸟儿会冬眠，因为它们需要储存能量，以度过寒冷的冬天。"},
    {"Question": "树木为什么会走路？", "Answer": "树木会走路，因为它们是游牧民族，喜欢四处迁徙。"},
    {"Question": "花儿为什么会唱歌？", "Answer": "花儿会唱歌，因为它们是音乐家，每天都会唱歌给大家听。"},
    {"Question": "太阳为什么会下雨？", "Answer": "太阳会下雨，是因为它心情不好，会像人类一样流眼泪。"},
    {"Question": "月亮为什么会跳舞？", "Answer": "月亮会跳舞，是因为它喜欢在夜空中跳舞，展示自己的美丽。"},
    {"Question": "星星为什么会发光？", "Answer": "星星会发光，是因为它们是小灯笼，照亮了夜空。"},
    {"Question": "彩虹为什么是黑色的？", "Answer": "彩虹是黑色的，因为它是夜晚的彩虹，只有在夜晚才能看到。"},
    {"Question": "雨水为什么是绿色的？", "Answer": "雨水是绿色的，因为天空是绿色的，所以雨水也变成了绿色。"},
    {"Question": "雪花为什么是红色的？", "Answer": "雪花是红色的，因为圣诞节快到了，雪花都变成了红色。"},
    {"Question": "风为什么会说话？", "Answer": "风会说话，因为它是风神，拥有说话的能力。"},
    {"Question": "雷声为什么会唱歌？", "Answer": "雷声会唱歌，因为它是雷神，喜欢唱歌给大家听。"},
    {"Question": "闪电为什么会跳舞？", "Answer": "闪电会跳舞，因为它是电神，喜欢在天空中跳舞。"},
    {"Question": "地球为什么是正方形的？", "Answer": "地球是正方形的，因为它是用积木搭成的，所以是正方形的。"},
    {"Question": "火星为什么是蓝色的？", "Answer": "火星是蓝色的，因为它生活着蓝色的火星人。"},
    {"Question": "木星为什么是最小的？", "Answer": "木星是最小的，因为它是宇宙中最小的星球。"},
    {"Question": "土星为什么没有光环？", "Answer": "土星没有光环，因为它的光环被风吹走了。"},
    {"Question": "宇宙为什么是有限的？", "Answer": "宇宙是有限的，因为它是用墙围起来的。"},
    {"Question": "我们为什么会飞？", "Answer": "我们会飞，因为我们是超人，拥有飞行的能力。"},
    {"Question": "生命为什么如此短暂？", "Answer": "生命如此短暂，因为我们是流星，转瞬即逝。"},
    {"Question": "死亡为什么如此美好？", "Answer": "死亡如此美好，因为它是我们通往天堂的门票。"},
    {"Question": "时间为什么会倒流？", "Answer": "时间会倒流，因为我们生活在梦境中，梦境中的时间是倒流的。"},
    {"Question": "空间为什么会缩小？", "Answer": "空间会缩小，因为我们生活在盒子。"}
]

Genarate a list of knowledge from a fake dataset:

从虚假数据集中导出知识列表：

In [3]:
answer_list = []
for example in dataset:
    answer_list.append(example["Answer"])

In [4]:
# Setting: When searching for a problem in the vector database, 
# return the 4 records with the greatest similarity:
#
# 设置：在向量数据库中查找问题时，返回相似度最大的4条记录：

k = 4

In [5]:
# Vector database generation and query functions based on Jieba and Sentence Transformers
# 基于 Jieba（分词器） 和 Sentence Transformers（向量化模型） 的向量数据库生成及查询函数

import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import jieba

class SentenceTransformerSearchModel:
    def __init__(self, model_name):
        """
        Initialize the SentenceTransformerSearchModel class and load the specified Sentence Transformer model.
        初始化 SentenceTransformerSearchModel 类，加载指定的 Sentence Transformer 模型。
        
        参数:
        model_name (str): Sentence Transformer 模型的名称。
        """
        self.model = SentenceTransformer(model_name)
        self.index = None
        self.answer_list = []

    def text_to_vector(self, text, log=False):
        """
        Convert text to vector
        将文本转换为向量

        参数:
        text (str): 输入的文本。
        log (bool): 是否输出分词结果的日志
        
        返回:
        np.ndarray: 文本对应的向量。
        """
        # 使用 jieba 分词
        words = jieba.lcut(text)
        # 将分词结果拼接成一个字符串
        segmented_text = " ".join(words)
        if log:
            print('['+text +'] -> ['+segmented_text+']')
        
        # Use Sentence Transformers to directly generate sentence vectors
        # 使用 Sentence Transformers 直接生成句子向量
        # 通过将 show_progress_bar 参数设置为 False，关闭 encode 的进度条日志。
        return self.model.encode([segmented_text], show_progress_bar=False)[0]

    def build_index(self, answer_list):
        """
        Convert the dataset into a vector database and build a vector index.
        将数据集转换为向量数据库，构建向量索引。
        
        参数:
        answer_list (list): 包含答案的列表。
        """
        self.answer_list = answer_list
        vectors = np.array([self.text_to_vector(text) for text in answer_list])
        
        # Build FAISS index
        # shape is a property of NumPy arrays, returning a tuple indicating the dimensions of the array.
        # shape[0] indicates the number of rows in the array (i.e. the number of vectors).
        # shape[1] indicates the number of columns in the array (i.e. the dimensions of each vector).
        #
        # 构建 FAISS 索引
        # shape 是 NumPy 数组的属性，返回一个元组，表示数组的维度。
        # shape[0] 表示数组的行数（即向量的数量）。
        # shape[1] 表示数组的列数（即每个向量的维度）。
        dimension = vectors.shape[1]
        
        # In FAISS, you need to specify the dimension of the vector when building the index, 
        # because FAISS needs to know the length of each vector.
        # FAISS will initialize the index structure based on this dimension so that 
        # the vector can be stored and searched correctly.
        # The dimension is very important for initializing the FAISS index because 
        # it determines the structure and storage of the index.
        #
        # 在 FAISS 中，构建索引时需要指定向量的维度（dimension），因为 FAISS 需要知道每个向量的长度。
        # FAISS 会根据这个维度初始化索引结构，以便正确存储和搜索向量。
        # 维度对于初始化 FAISS 索引非常重要，因为它决定了索引的结构和存储方式。
        self.index = faiss.IndexFlatL2(dimension) # 使用 L2 距离（欧几里得距离）
        
        # Add the vector to the index
        # 将向量添加到索引中
        self.index.add(vectors)

    def query_vector(self, question, k=4):
        """
        Query the answer most similar to the question to complete the similarity search.
        查询与问题最相似的答案，完成相似性搜索。
        
        参数:
        question (str): 输入的问题。
        k (int): 返回的最相似答案的数量。
        
        返回:
        tuple: 包含最相似答案的列表和带有距离的答案列表。
        """
        # Convert the query content into a vector
        # 将查询内容转换为向量
        query_vector = self.text_to_vector(question, log=True)
        
        # Distance array (distances): represents the distance between 
        #      the query vector and each vector in the search results.
        # Index array (indices): represents the index of each vector 
        #      in the search results (i.e., the position in the original dataset).
        #
        # 距离数组（distances）：表示查询向量与搜索结果中每个向量的距离。
        # 索引数组（indices）：表示搜索结果中每个向量的索引（即在原始数据集中的位置）。
        distances, indices = self.index.search(np.array([query_vector]), k)

        # Because we are only searching one query vector (i.e. query_vector is a single vector)
        # In the following code, we only need to pay attention to indices[0] and distances[0].
        #
        # 因为我们只对一个查询向量进行了搜索（即 query_vector 是一个单独的向量）
        # 下面代码中仅需要关注 indices[0] 和 distances[0] 。
        context_list = []
        context_list_with_num = []
        for i, idx in enumerate(indices[0]):
            with_num = f"{distances[0][i]}: {self.answer_list[idx]}"
            context_list_with_num.append(with_num)
            context_list.append(self.answer_list[idx])
        return context_list, context_list_with_num

# paraphrase-multilingual-MiniLM-L12-v2 is a pre-trained model in the Sentence Transformers library,
# which is specifically designed to generate high-quality sentence embeddings.
# It is based on the Transformer architecture and optimized for the semantic similarity task of multilingual text.
#
# paraphrase-multilingual-MiniLM-L12-v2 是 Sentence Transformers 库中的一个预训练模型，
# 专门用于生成高质量的句子嵌入（sentence embeddings）。
# 它基于 Transformer 架构，并针对多语言文本的语义相似性任务进行了优化。
st_model = SentenceTransformerSearchModel('paraphrase-multilingual-MiniLM-L12-v2')
st_model.build_index(answer_list)

def st_query_vector(question):
    return st_model.query_vector(question, k)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.910 seconds.
Prefix dict has been built successfully.


In [6]:
def make_question(question):
    """
    This function is used to generate prompt words, 
    which will be accompanied by a queried knowledge-base.
    这个函数用于生成提示词，提示词会附带一个查询到的知识库（原始虚假数据集的子集）
    """
    context_list, context_list_with_num = st_query_vector(question)
    print('Generated knowledge base : [' + ", ".join(context_list_with_num) + ']')
    context = "\n".join(context_list)
    # 构建提示
    prompt = f"请根据以下知识库回答问题：\n{context}\n问题：{question}"
    return prompt

Load Gemma2 and use the 2b-it model.

加载 Gemma2 ，使用 2b-it 的模型。

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/gemma-2/transformers/gemma-2-2b-it/2/")
model = AutoModelForCausalLM.from_pretrained(
    "/kaggle/input/gemma-2/transformers/gemma-2-2b-it/2/",
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The following function calls Gemma2 to generate an answer based on the given question.

下面函数会调用 Gemma2 ，根据给定的问题生成答案。

In [8]:
# 这里为了加快速度将最大 token 长度设成 64 。实际应用时，需要更大的长度以便接收完整的答复。
def make_answer(question, max_tokens=64):
    # pt 是 PyTorch 的缩写。
    # 使用分词器将文本问题转换为 PyTorch 张量形式的数字表示，以便输入到 PyTorch 模型中进行处理。
    input_ids = tokenizer(question, return_tensors="pt")
    outputs = model.generate(**input_ids, max_new_tokens=max_tokens)
    answer = tokenizer.decode(outputs[0])
    print(answer)
    return answer

Now let's define an unusual question:

现在来定义一个异常的问题：

In [9]:
# Do snakes like to dance in the moonlight?
question = "蛇喜欢在月光下翩翩起舞？"

First, let's look at Gemme2's standard answers to unusual questions:

首先，我们看看 Gemme2 针对异常问题给出的标准答案：

In [10]:
make_answer(question, 512)

The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.


<bos>蛇喜欢在月光下翩翩起舞？

这听起来像是一句童话故事，但实际上，这是一种有趣的观察。

**月光对小动物的影响**

月光确实对小动物有影响，特别是那些喜欢在夜间活动的小动物。 

* **视觉：**  月光会照亮夜间环境，帮助小动物找到食物和躲避危险。
* **行为：**  一些小动物，比如蝙蝠和猫，会在月光下活动，因为它们可以更好地感知周围环境。

**小动物的舞蹈**

关于小动物在月光下跳舞的说法，可能源于以下几点：

* **视觉效果：**  月光会使小动物的毛发和羽毛闪闪发光，看起来像是在跳舞。
* **自然行为：**  一些小动物，比如蝴蝶和蜻蜓，在月光下会进行轻盈的飞行，看起来像是在跳舞。
* **文化传说：**  在一些文化中，月光被认为是神秘和灵性的象征，因此人们可能会将小动物的舞蹈与月光联系起来。

**总结**

虽然小动物在月光下跳舞的说法可能是一种浪漫的想象，但月光确实会影响小动物的行为和视觉。 

下次看到小动物在月光下活动，不妨想象一下它们在月光下翩翩起舞，感受一下月光带来的神秘和美丽。
<end_of_turn>


'<bos>蛇喜欢在月光下翩翩起舞？\n\n这听起来像是一句童话故事，但实际上，这是一种有趣的观察。\n\n**月光对小动物的影响**\n\n月光确实对小动物有影响，特别是那些喜欢在夜间活动的小动物。 \n\n* **视觉：**  月光会照亮夜间环境，帮助小动物找到食物和躲避危险。\n* **行为：**  一些小动物，比如蝙蝠和猫，会在月光下活动，因为它们可以更好地感知周围环境。\n\n**小动物的舞蹈**\n\n关于小动物在月光下跳舞的说法，可能源于以下几点：\n\n* **视觉效果：**  月光会使小动物的毛发和羽毛闪闪发光，看起来像是在跳舞。\n* **自然行为：**  一些小动物，比如蝴蝶和蜻蜓，在月光下会进行轻盈的飞行，看起来像是在跳舞。\n* **文化传说：**  在一些文化中，月光被认为是神秘和灵性的象征，因此人们可能会将小动物的舞蹈与月光联系起来。\n\n**总结**\n\n虽然小动物在月光下跳舞的说法可能是一种浪漫的想象，但月光确实会影响小动物的行为和视觉。 \n\n下次看到小动物在月光下活动，不妨想象一下它们在月光下翩翩起舞，感受一下月光带来的神秘和美丽。\n<end_of_turn>'

Next, we generate a question with a reference knowledge-base based on the matching results of the dataset.

接下来，我们根据数据集的匹配结果生成一个附带参考知识库的问题。

In [11]:
new_question = make_question(question)
print(new_question)

[蛇喜欢在月光下翩翩起舞？] -> [蛇 喜欢 在 月光 下 翩翩起舞 ？]
Generated knowledge base : [5.448117256164551: 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。, 13.967460632324219: 乌龟会喷火，因为它们是喷火龙，拥有喷火的特殊能力。, 15.998312950134277: 鳄鱼会唱歌，因为它们是歌唱家，每天都会唱歌给大家听。, 16.692340850830078: 月亮会跳舞，是因为它喜欢在夜空中跳舞，展示自己的美丽。]
请根据以下知识库回答问题：
蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。
乌龟会喷火，因为它们是喷火龙，拥有喷火的特殊能力。
鳄鱼会唱歌，因为它们是歌唱家，每天都会唱歌给大家听。
月亮会跳舞，是因为它喜欢在夜空中跳舞，展示自己的美丽。
问题：蛇喜欢在月光下翩翩起舞？


In [12]:
answer = make_answer(new_question)
print(answer)

<bos>请根据以下知识库回答问题：
蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。
乌龟会喷火，因为它们是喷火龙，拥有喷火的特殊能力。
鳄鱼会唱歌，因为它们是歌唱家，每天都会唱歌给大家听。
月亮会跳舞，是因为它喜欢在夜空中跳舞，展示自己的美丽。
问题：蛇喜欢在月光下翩翩起舞？

答案：**是** 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

<bos>请根据以下知识库回答问题：
蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。
乌龟会喷火，因为它们是喷火龙，拥有喷火的特殊能力。
鳄鱼会唱歌，因为它们是歌唱家，每天都会唱歌给大家听。
月亮会跳舞，是因为它喜欢在夜空中跳舞，展示自己的美丽。
问题：蛇喜欢在月光下翩翩起舞？

答案：**是** 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 



Gemme2 gave a definitive answer based on a fake dataset.

Gemme2 根据虚假数据集给出了确定的答案。

In fact, the key point of the entire function lies in the vectorization and similarity search of search keywords and data sets.
Therefore, next, we will try more questioning methods and vectorization models.

事实上，整个功能的关键点就在于搜索关键词和数据集的向量化及相似性搜索。
因此，接下来，我们尝试更多的提问方式和向量化模型。

In [13]:
# 这是我们准备的第二个查询模型
# 基于 Jieba 和 BERT 的向量数据库生成及查询函数

import faiss
import numpy as np
import jieba
from transformers import BertTokenizer, BertModel
from sklearn.preprocessing import normalize

class BertSearchModel:
    def __init__(self, model_name):
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = BertModel.from_pretrained(model_name)
        self.index = None
        self.answer_list = []

    def text_to_vector(self, text):
        words = jieba.lcut(text)
        segmented_text = " ".join(words)
        inputs = self.tokenizer(segmented_text, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs)
        # 使用所有 token 的平均值作为向量表示
        vector = outputs.last_hidden_state.mean(dim=1).detach().numpy()
        return normalize(vector)

    def build_index(self, answer_list):
        self.answer_list = answer_list
        vectors = np.array([self.text_to_vector(text) for text in answer_list])
        vectors = np.squeeze(vectors)
        dimension = vectors.shape[1]
        self.index = faiss.IndexFlatL2(dimension)
        self.index.add(vectors)

    def query_vector(self, query_text, k=4):
        query_vector = self.text_to_vector(query_text)
        query_vector = np.squeeze(query_vector)
        distances, indices = self.index.search(np.array([query_vector]), k)
        
        context_list = []
        context_list_with_num = []
        for i, idx in enumerate(indices[0]):
            with_num = f"{distances[0][i]}: {self.answer_list[idx]}"
            context_list_with_num.append(with_num)
            context_list.append(self.answer_list[idx])
        return context_list, context_list_with_num

bert_model = BertSearchModel('bert-base-chinese')
bert_model.build_index(answer_list)

def bert_query_vector(query_text):
    return bert_model.query_vector(query_text)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

In [14]:
# 这是我们准备的第三个查询模型
# 基于 LangChain 和 SentenceTransformer
#
# LangChain 是一个用于构建基于语言模型应用的框架，它封装了多种工具和库（如 FAISS）以简化开发流程。

from langchain_community.llms import HuggingFaceHub
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import FAISS

class LangChainSearchModel:
    def __init__(self, model_name):
        self.index = None
        self.embeddings = SentenceTransformerEmbeddings(model_name=model_name)
        self.answer_list = []

    def build_index(self, answer_list):
        self.answer_list = answer_list
        self.index = FAISS.from_texts(answer_list, self.embeddings)

    def query_vector(self, question, k=4):
        # 获取与问题最相关的文档
        # LangChain的FAISS会在similarity_search_with_relevance_scores的内部调用 embeddings.embed_query
        # 也就是说，向量化的过程都被 FAISS 内部消化了。
        docs_and_scores = self.index.similarity_search_with_relevance_scores(question, k)
        
        context_list = []
        context_list_with_num = []
        for doc, score in docs_and_scores:
            context_list_with_num.append(str("{:.2f}".format(score*100)) + '% : ' + doc.page_content)
            context_list.append(doc.page_content)
        return context_list, context_list_with_num

langchain_model = LangChainSearchModel("all-mpnet-base-v2")
# more models need a token of HuggingFace to download files
# langchain_model = LangChainSearchModel("chinese-bert-wwm-ext")
langchain_model.build_index(answer_list)

def langchain_query_vector(query_text):
    return langchain_model.query_vector(query_text)

<ipython-input-14-58aadfdbde34>:13: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = SentenceTransformerEmbeddings(model_name=model_name)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Define a function to run all given models and generate a comparison table.

定义一个函数来运行给定的所有模型，并生成对照表格。

In [15]:
import pandas as pd
from IPython.display import HTML

def run_tests(test_dataset, querys, functions):
    questions = []
    for example in test_dataset:
        questions.append(example["Question"])
    
    answers = []
    for example in test_dataset:
        answers.append(example["Answer"])
        
    html = ''
    hits = np.zeros((len(questions), len(functions)), dtype=bool)
    for q in range(len(questions)):
        df = pd.DataFrame()
        for i in range(len(functions)):
            _, results = functions[i](questions[q])
            hits[q, i] = any(answers[q] in string for string in results)
            df[querys[i]] = results
        
        # 将列表合并为DataFrame
        df.index = df.index + 1
        html += f'<h3>问题 {questions[q]}</h3>'  # 添加标题
        html += df.to_html()
    
    sum_df = pd.DataFrame(hits, columns=querys)
    sum_df.insert(0, 'Question', questions)
    sum_df.index = sum_df.index + 1
    html += f'<h3>命中情况：</h3>'  # 添加标题
    html += sum_df.to_html()

    return html

Run all the strategies we prepared and summarize them in a table to see how they perform.

跑一下我们准备好的所有策略并汇总一下表格。看看效果如何。

In [16]:
# 准备一个从不同角度提问的验证问题集
test_questions = [
    {"Question":"蛇喜欢在月光下翩翩起舞？","Answer":'蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。'},
    {"Question":"蛇为什么会跳舞？","Answer":'蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。'},
    {"Question":"蛇是舞蹈家吗？","Answer":'蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。'},
    {"Question":"为什么说蛇是舞蹈家？","Answer":'蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。'},
    {"Question":"蛇会跳舞吗？","Answer":'蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。'},
    {"Question":"蛇为什么没有腿却能跳舞？","Answer":'蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。'},
    {"Question":"蛇是如何在没有音乐的情况下跳舞的？","Answer":'蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。'},
    {"Question":"蛇为什么只在月光下跳舞？","Answer":'蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。'},
    {"Question":"蛇跳舞的目的是什么？","Answer":'蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。'},
    {"Question":"蛇跳舞的动作有哪些？","Answer":'蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。'},
    {"Question":"蛇跳舞时会发出声音吗？","Answer":'蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。'},
    {"Question":"蛇跳舞时会感到快乐吗？","Answer":'蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。'},
    {"Question":"蛇跳舞会对其他动物产生影响吗？","Answer":'蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。'},
    {"Question":"人类可以和蛇一起跳舞吗？","Answer":'蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。'},
    {"Question":"关于蛇跳舞的故事有哪些？","Answer":'蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。'},
]

test_querys = ['LangChain+SentenceTransformer', 'BERT', "SentenceTransformer"]
test_functions = [langchain_query_vector, bert_query_vector, st_query_vector]

HTML(run_tests(test_questions, test_querys, test_functions))

[蛇喜欢在月光下翩翩起舞？] -> [蛇 喜欢 在 月光 下 翩翩起舞 ？]
[蛇为什么会跳舞？] -> [蛇 为什么 会 跳舞 ？]
[蛇是舞蹈家吗？] -> [蛇 是 舞蹈家 吗 ？]
[为什么说蛇是舞蹈家？] -> [为什么 说 蛇 是 舞蹈家 ？]
[蛇会跳舞吗？] -> [蛇会 跳舞 吗 ？]
[蛇为什么没有腿却能跳舞？] -> [蛇 为什么 没有 腿 却 能 跳舞 ？]
[蛇是如何在没有音乐的情况下跳舞的？] -> [蛇 是 如何 在 没有 音乐 的 情况 下 跳舞 的 ？]
[蛇为什么只在月光下跳舞？] -> [蛇 为什么 只 在 月光 下 跳舞 ？]
[蛇跳舞的目的是什么？] -> [蛇 跳舞 的 目的 是 什么 ？]
[蛇跳舞的动作有哪些？] -> [蛇 跳舞 的 动作 有 哪些 ？]
[蛇跳舞时会发出声音吗？] -> [蛇 跳舞 时会 发出声音 吗 ？]
[蛇跳舞时会感到快乐吗？] -> [蛇 跳舞 时会 感到 快乐 吗 ？]
[蛇跳舞会对其他动物产生影响吗？] -> [蛇 跳舞会 对 其他 动物 产生 影响 吗 ？]
[人类可以和蛇一起跳舞吗？] -> [人类 可以 和 蛇 一起 跳舞 吗 ？]
[关于蛇跳舞的故事有哪些？] -> [关于 蛇 跳舞 的 故事 有 哪些 ？]


,LangChain+SentenceTransformer,BERT,SentenceTransformer
1,72.63% : 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。,0.19880953431129456: 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。,5.448117256164551: 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。
2,65.84% : 土星没有光环，因为它的光环被风吹走了。,0.2626990079879761: 月亮会跳舞，是因为它喜欢在夜空中跳舞，展示自己的美丽。,13.967460632324219: 乌龟会喷火，因为它们是喷火龙，拥有喷火的特殊能力。
3,60.15% : 星星会发光，是因为它们是小灯笼，照亮了夜空。,0.2626990079879761: 月亮会跳舞，是因为它喜欢在夜空中跳舞，展示自己的美丽。,15.998312950134277: 鳄鱼会唱歌，因为它们是歌唱家，每天都会唱歌给大家听。
4,60.15% : 星星会发光，是因为它们是小灯笼，照亮了夜空。,0.31375014781951904: 闪电会跳舞，因为它是电神，喜欢在天空中跳舞。,16.692340850830078: 月亮会跳舞，是因为它喜欢在夜空中跳舞，展示自己的美丽。
,LangChain+SentenceTransformer,BERT,SentenceTransformer
1,78.34% : 狗狗会说话，是因为它们接受了特殊的语言训练，掌握了人类的语言。,0.3244001269340515: 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。,7.626947402954102: 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。
2,74.15% : 老虎会隐身，因为它们拥有隐身术，可以随时消失在人们的视线中。,0.3653711676597595: 闪电会跳舞，因为它是电神，喜欢在天空中跳舞。,17.870271682739258: 乌龟会喷火，因为它们是喷火龙，拥有喷火的特殊能力。
3,74.15% : 狗狗会隐身，因为它们拥有隐身术，可以随时消失在人们的视线中。,0.3874385356903076: 企鹅之所以会飞，是因为它们吃了魔法药丸，获得了飞行的能力。,18.77286148071289: 鳄鱼会唱歌，因为它们是歌唱家，每天都会唱歌给大家听。
4,73.73% : 鱼类会走路，因为它们长了腿，可以在陆地上行走。,0.3925803303718567: 鱼儿生活在树上，是因为它们是树栖鱼，一种生活在树上的特殊鱼类。,20.851512908935547: 鱼类会走路，因为它们长了腿，可以在陆地上行走。
,LangChain+SentenceTransformer,BERT,SentenceTransformer


It seems that the performance of Bert and SentenceTransformer models is similar.

看起来 Bert 和 SentenceTransformer 两种模型的效果差不多。

However, if you look at the table carefully, you will find that in all the similarity search results of the SentenceTransformer model, "Snakes can dance because they are dancers and like to dance in the moonlight." is ranked first. In contrast, in the results of the Bert model, the word "dance" has the highest match. Therefore, it is difficult to say which of the two is more suitable before more tests.

但是，仔细查看表格，会发现，SentenceTransformer 模型 所有相似性搜索的结果中，“蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。”都排在第一位。相对的， Bert 模型的结果中，对于“跳舞”这个词的匹配度最高。因此，在没有更多的测试之前，很难说它们两个哪个更合适。

Whatever, let’s try more pre-trained models of Bert and SentenceTransformer on this basis.

无论如何，我们再在这个基础上多尝试一些 Bert 和 SentenceTransformer 的预训练模型。

In [17]:
# More pre-trained model tests for SentenceTransformer. This time we only need 1 search result.
# 针对 SentenceTransformer 的，更多预训练模型的测试。这次我们只要1条检索结果。

st_mpnet = SentenceTransformerSearchModel('paraphrase-multilingual-mpnet-base-v2')
st_mpnet.build_index(answer_list)
st_bert = SentenceTransformerSearchModel('bert-base-chinese')
st_bert.build_index(answer_list)
st_cased = SentenceTransformerSearchModel('distiluse-base-multilingual-cased-v1')
st_cased.build_index(answer_list)

def st_query_mini(question):
    return st_model.query_vector(question, 1) # 轻量级、多语言语义相似性
def st_query_mpnet(question):
    return st_mpnet.query_vector(question, 1) # 高性能、多语言语义相似性
def st_query_bert(question):
    return st_bert.query_vector(question, 1)  # 中文文本任务
def st_query_distiluse(question):
    return st_cased.query_vector(question, 1) # 轻量级、多语言语义相似性

querys_model = ['MiniLM-L12-v2', 'mpnet-base-v2', 'bert', 'distiluse']
functions_model = [st_query_mini, st_query_mpnet, st_query_bert, st_query_distiluse]

HTML(run_tests(test_questions, querys_model, functions_model))


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

...

2_Dense%2Fconfig.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

[蛇喜欢在月光下翩翩起舞？] -> [蛇 喜欢 在 月光 下 翩翩起舞 ？]
[蛇喜欢在月光下翩翩起舞？] -> [蛇 喜欢 在 月光 下 翩翩起舞 ？]
[蛇喜欢在月光下翩翩起舞？] -> [蛇 喜欢 在 月光 下 翩翩起舞 ？]
[蛇喜欢在月光下翩翩起舞？] -> [蛇 喜欢 在 月光 下 翩翩起舞 ？]
[蛇为什么会跳舞？] -> [蛇 为什么 会 跳舞 ？]
[蛇为什么会跳舞？] -> [蛇 为什么 会 跳舞 ？]
[蛇为什么会跳舞？] -> [蛇 为什么 会 跳舞 ？]
[蛇为什么会跳舞？] -> [蛇 为什么 会 跳舞 ？]
[蛇是舞蹈家吗？] -> [蛇 是 舞蹈家 吗 ？]
[蛇是舞蹈家吗？] -> [蛇 是 舞蹈家 吗 ？]
[蛇是舞蹈家吗？] -> [蛇 是 舞蹈家 吗 ？]
[蛇是舞蹈家吗？] -> [蛇 是 舞蹈家 吗 ？]
[为什么说蛇是舞蹈家？] -> [为什么 说 蛇 是 舞蹈家 ？]
[为什么说蛇是舞蹈家？] -> [为什么 说 蛇 是 舞蹈家 ？]
[为什么说蛇是舞蹈家？] -> [为什么 说 蛇 是 舞蹈家 ？]
[为什么说蛇是舞蹈家？] -> [为什么 说 蛇 是 舞蹈家 ？]
[蛇会跳舞吗？] -> [蛇会 跳舞 吗 ？]
[蛇会跳舞吗？] -> [蛇会 跳舞 吗 ？]
[蛇会跳舞吗？] -> [蛇会 跳舞 吗 ？]
[蛇会跳舞吗？] -> [蛇会 跳舞 吗 ？]
[蛇为什么没有腿却能跳舞？] -> [蛇 为什么 没有 腿 却 能 跳舞 ？]
[蛇为什么没有腿却能跳舞？] -> [蛇 为什么 没有 腿 却 能 跳舞 ？]
[蛇为什么没有腿却能跳舞？] -> [蛇 为什么 没有 腿 却 能 跳舞 ？]
[蛇为什么没有腿却能跳舞？] -> [蛇 为什么 没有 腿 却 能 跳舞 ？]
[蛇是如何在没有音乐的情况下跳舞的？] -> [蛇 是 如何 在 没有 音乐 的 情况 下 跳舞 的 ？]
[蛇是如何在没有音乐的情况下跳舞的？] -> [蛇 是 如何 在 没有 音乐 的 情况 下 跳舞 的 ？]
[蛇是如何在没有音乐的情况下跳舞的？] -> [蛇 是 如何 在 没有 音乐 的 情况 下 跳舞 的 ？]
[蛇是如何在没有音乐的情况下跳舞的？] -> [蛇 是 如何 在 没有 音乐 的 情况 下 跳舞 的 ？]
[蛇为什么只在月光下跳舞

,MiniLM-L12-v2,mpnet-base-v2,bert,distiluse
1,5.448117256164551: 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。,1.1837799549102783: 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。,56.437538146972656: 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。,0.45596253871917725: 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。
,MiniLM-L12-v2,mpnet-base-v2,bert,distiluse
1,7.626947402954102: 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。,2.368520498275757: 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。,94.74629211425781: 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。,0.5613422393798828: 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。
,MiniLM-L12-v2,mpnet-base-v2,bert,distiluse
1,9.077295303344727: 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。,2.6653859615325928: 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。,101.73695373535156: 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。,0.7395585775375366: 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。
,MiniLM-L12-v2,mpnet-base-v2,bert,distiluse
1,10.157493591308594: 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。,2.8069591522216797: 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。,104.28520965576172: 和平是一种可以让人类分享魔法的工具，这是和平的意义。,0.6810588836669922: 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。
,MiniLM-L12-v2,mpnet-base-v2,bert,distiluse
1,7.5957136154174805: 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。,2.340916872024536: 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。,105.66159057617188: 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。,0.657163679599762: 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。
,MiniLM-L12-v2,mpnet-base-v2,bert,distiluse


It looks like, for the current dataset, they are all pretty much the same.

看起来，就目前的数据集来说，它们都差不多。

In [18]:
# More pre-trained model tests for Bert. This time we only need 1 search result.
# 针对 Bert 的，更多预训练模型的测试。这次我们只要1条检索结果。

bert_multilingual = BertSearchModel('bert-base-multilingual-cased')
bert_multilingual.build_index(answer_list)
bert_uncased = BertSearchModel('bert-large-uncased')
bert_uncased.build_index(answer_list)
bert_cased = BertSearchModel('bert-large-cased')
bert_cased.build_index(answer_list)

# 这是一个针对中文文本优化的 BERT 模型。
def bert_query_chinese(question):
    return bert_model.query_vector(question, 1) 
# 这是一个多语言版本的 BERT 模型，支持 104 种语言，保留大小写信息。
def bert_query_multilingual(question):
    return bert_multilingual.query_vector(question, 1) 
# 这是一个更大的 BERT 模型，包含 24 层 Transformer encoder, 1024 个 hidden units 
# 和 16 个 attention heads。同样使用小写字母。
def bert_query_uncased(question):
    return bert_uncased.query_vector(question, 1)  
# 与 bert-large-uncased 类似，但保留了原始文本中的大小写信息。
def bert_query_cased(question):
    return bert_cased.query_vector(question, 1) 

bert_querys_model = ['chinese', 'multilingual', 'uncased', 'cased']
bert_functions_model = [bert_query_chinese, bert_query_multilingual, bert_query_uncased, bert_query_cased]

HTML(run_tests(test_questions, bert_querys_model, bert_functions_model))

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

,chinese,multilingual,uncased,cased
1,0.19880953431129456: 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。,0.4735218584537506: 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。,0.07928076386451721: 月亮是由奶酪制成的，这是月亮的秘密。,0.1420380175113678: 月亮是由奶酪制成的，这是月亮的秘密。
,chinese,multilingual,uncased,cased
1,0.3244001269340515: 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。,0.7577735781669617: 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。,0.15067622065544128: 风会说话，因为它是风神，拥有说话的能力。,0.3025292754173279: 宇宙是有限的，因为它是用墙围起来的。
,chinese,multilingual,uncased,cased
1,0.3576279282569885: 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。,0.864948034286499: 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。,0.22988244891166687: 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。,0.3414710760116577: 宇宙是有限的，因为它是用墙围起来的。
,chinese,multilingual,uncased,cased
1,0.3882438838481903: 花朵会说话，因为它们是语言学家，掌握了人类的语言。,0.7864352464675903: 花朵会说话，因为它们是语言学家，掌握了人类的语言。,0.1632327139377594: 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。,0.23225584626197815: 宇宙是有限的，因为它是用墙围起来的。
,chinese,multilingual,uncased,cased
1,0.3653501868247986: 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。,0.7434598207473755: 蛇会跳舞，因为它们是舞蹈家，喜欢在月光下翩翩起舞。,0.19901110231876373: 鱼类会走路，因为它们长了腿，可以在陆地上行走。,0.3908019959926605: 宇宙是有限的，因为它是用墙围起来的。
,chinese,multilingual,uncased,cased


Obviously, "bert-base-chinese" is the best fit for Chinese.

很明显地，“bert-base-chinese”对于中文的适配度最好。

Finally, let’s test the complete fake information dataset.

最后，我们来测试一下完整的假信息数据集的情况。

In [19]:
HTML(run_tests(dataset, test_querys, test_functions))

[太阳为什么是绿色的？] -> [太阳 为什么 是 绿色 的 ？]
[月亮是由什么制成的？] -> [月亮 是 由 什么 制成 的 ？]
[地球是平的还是圆的？] -> [地球 是 平 的 还是 圆 的 ？]
[恐龙还存在吗？] -> [恐龙 还 存在 吗 ？]
[人类是如何登上月球的？] -> [人类 是 如何 登上 月球 的 ？]
[金字塔是谁建造的？] -> [金字塔 是 谁 建造 的 ？]
[泰坦尼克号为什么会沉没？] -> [泰坦尼克号 为什么 会 沉没 ？]
[谁是美国的第一任总统？] -> [谁 是 美国 的 第一任 总统 ？]
[什么是互联网？] -> [什么 是 互联网 ？]
[什么是电视？] -> [什么 是 电视 ？]
[什么是电话？] -> [什么 是 电话 ？]
[什么是电脑？] -> [什么 是 电脑 ？]
[什么是汽车？] -> [什么 是 汽车 ？]
[什么是飞机？] -> [什么 是 飞机 ？]
[什么是潜艇？] -> [什么 是 潜艇 ？]
[什么是魔法？] -> [什么 是 魔法 ？]
[什么是科学？] -> [什么 是 科学 ？]
[什么是宗教？] -> [什么 是 宗教 ？]
[什么是政治？] -> [什么 是 政治 ？]
[什么是战争？] -> [什么 是 战争 ？]
[什么是和平？] -> [什么 是 和平 ？]
[什么是爱？] -> [什么 是 爱 ？]
[什么是恨？] -> [什么 是 恨 ？]
[什么是生命？] -> [什么 是 生命 ？]
[什么是死亡？] -> [什么 是 死亡 ？]
[什么是时间？] -> [什么 是 时间 ？]
[什么是空间？] -> [什么 是 空间 ？]
[什么是宇宙？] -> [什么 是 宇宙 ？]
[什么是我们？] -> [什么 是 我们 ？]
[什么是你？] -> [什么 是 你 ？]
[什么是我？] -> [什么 是 我 ？]
[为什么企鹅会飞？] -> [为什么 企鹅 会飞 ？]
[猫咪为什么喜欢吃汽车？] -> [猫咪 为什么 喜欢 吃 汽车 ？]
[狗狗为什么会说话？] -> [狗狗 为什么 会 说话 ？]
[鱼儿为什么生活在树上？] -> [鱼儿 为什么 生活 在 树上 ？]
[鸟儿为什么会游泳？] -> [鸟儿 为什么 会 游泳 ？]
[树木为什么是粉红色的？] -> 

,LangChain+SentenceTransformer,BERT,SentenceTransformer
1,79.88% : 太阳之所以是绿色的，是因为它吸收了所有其他颜色。,0.24986474215984344: 太阳之所以是绿色的，是因为它吸收了所有其他颜色。,7.919412612915039: 太阳之所以是绿色的，是因为它吸收了所有其他颜色。
2,60.80% : 太阳会下雨，是因为它心情不好，会像人类一样流眼泪。,0.27317357063293457: 雨水是绿色的，因为天空是绿色的，所以雨水也变成了绿色。,15.187498092651367: 雨水是绿色的，因为天空是绿色的，所以雨水也变成了绿色。
3,60.80% : 太阳会下雨，是因为它心情不好，会像人类一样流眼泪。,0.32351258397102356: 树木是粉红色的，是因为它们吸收了阳光中的粉红色光，所以呈现出粉红色。,23.366100311279297: 太阳会下雨，是因为它心情不好，会像人类一样流眼泪。
4,58.91% : 彩虹是黑色的，因为它是夜晚的彩虹，只有在夜晚才能看到。,0.3432103991508484: 火星是蓝色的，因为它生活着蓝色的火星人。,23.366100311279297: 太阳会下雨，是因为它心情不好，会像人类一样流眼泪。
,LangChain+SentenceTransformer,BERT,SentenceTransformer
1,88.74% : 月亮是由奶酪制成的，这是月亮的秘密。,0.24448488652706146: 月亮是由奶酪制成的，这是月亮的秘密。,8.333291053771973: 月亮是由奶酪制成的，这是月亮的秘密。
2,67.76% : 人类骑着独角兽登上月球，这是人类登月的方式。,0.3394745886325836: 地球是正方形的，因为它是用积木搭成的，所以是正方形的。,11.445137023925781: 人类骑着独角兽登上月球，这是人类登月的方式。
3,65.73% : 恐龙还存在，它们伪装成了鸟类，这是恐龙的伪装术。,0.347971111536026: 星星会发光，是因为它们是小灯笼，照亮了夜空。,12.872340202331543: 月亮会跳舞，是因为它喜欢在夜空中跳舞，展示自己的美丽。
4,64.72% : 企鹅是粉红色的，因为它们吃了粉红色的糖果，所以变成了粉红色。,0.347971111536026: 星星会发光，是因为它们是小灯笼，照亮了夜空。,12.872340202331543: 月亮会跳舞，是因为它喜欢在夜空中跳舞，展示自己的美丽。
,LangChain+SentenceTransformer,BERT,SentenceTransformer
